In [ ]:
from uau_api import UauAPI
from uau_api.settings import Settings
import functools
# Initialize the client
uau = UauAPI(Settings().API_URL, Settings().API_KEY)

In [ ]:
# Authenticate
uau.authenticate("leonardo", "hybr01")

In [ ]:
import inspect
import six

def inner_classes_list(cls):
    return [cls_attribute for cls_attribute in cls.__dict__.values()
            if inspect.isclass(cls_attribute)]

In [ ]:
dir(uau)

In [ ]:
empresa_obra_periodo = {"EmpresaObra": [
      {
        "Empresa": 33,
        "Obra": "ADM"
      }
    ],
    "PeriodoInicial": "2025-04-01T01:40:59.356Z",
    "PeriodoFinal": "2025-04-10T01:40:59.356Z"}
result = functools.reduce(getattr, ["ProcessoPagamento", "consultar_processos"], uau)(empresa_obra_periodo=empresa_obra_periodo)

In [ ]:
# Function to flatten nested dictionary with dot notation
def flatten_dict(d, parent_key='', sep='.'):
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)